<a href="https://colab.research.google.com/github/MagdaGielazyn/final_project_image_recognition/blob/master/MRI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<span style="color: lightblue">

# ML - PROJEKT ZALICZENIOWY
### 1. Project overview

<span>

In [ ]:
#import bibliotek:
import hashlib
import kagglehub
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import seaborn as sns
import pathlib
import pandas as pd
from PIL import Image
import tensorflow as tf
from tensorflow.keras import models, layers


In [ ]:
#pobierzmy dane na wypadek, gdyby autor zbioru dokonywal zmian

In [ ]:
#funkcje, ktore wrzucimy do folderku utils:

In [ ]:
#sprawdzenie balansu rozlozenia klas w zbiorze:
def check_balance(df, target):
    df_percentage = df[target].value_counts(normalize=True) * 100
    num = df_percentage.max()
    if num == 25:
        descr = "Dane są idealnie zrównoważone."
    elif num <= 35:
        descr = "Dane są dobrze zrównoważone."
    elif num <= 45:
        descr = "Lekka nierównowaga – akceptowalna."
    elif num <= 55:
        descr = "Umiarkowana nierównowaga – warto monitorować."
    else:
        descr = "Silna nierównowaga – zalecane balansowanie danych."
    return descr

#wczytanie danych:
def data_reader(path: str, img_size=(150, 150), batch_size=None):
    dataset_path = kagglehub.dataset_download(path)
    train_dir = os.path.join(dataset_path, "Training")
    test_dir = os.path.join(dataset_path, "Testing")
    train_ds = tf.keras.utils.image_dataset_from_directory(
        train_dir,
        image_size=img_size,
        batch_size=batch_size,
        label_mode="categorical",
        shuffle=False, ### na razie tak zamiast False zeby sciezki podpiac
    )
    test_ds = tf.keras.utils.image_dataset_from_directory(
        test_dir,
        image_size=img_size,
        batch_size=batch_size,
        label_mode="categorical",
        shuffle=False,
    )

    train_paths = tf.data.Dataset.from_tensor_slices(train_ds.file_paths)
    test_paths = tf.data.Dataset.from_tensor_slices(test_ds.file_paths)


    enriched_train_ds = tf.data.Dataset.zip((train_ds, train_paths))
    enriched_test_ds = tf.data.Dataset.zip((test_ds, test_paths))
    full_ds=enriched_train_ds.concatenate(enriched_test_ds)

    categories = set(train_ds.class_names+test_ds.class_names)
    #train_ds = train_ds.batch(batch_size)
    #test_ds = test_ds.batch(batch_size)

    return (full_ds, categories, train_ds, test_ds)

#zmiana datasetu na df:
def batched_dataset_to_df(ds):
    return pd.DataFrame([
        (p.numpy().decode(), int(np.argmax(y.numpy())))
        for (x, y), ps in ds
        for p, y in zip(ps, y)
    ], columns=["path", "label"])
def dataset_to_df(ds):
    return pd.DataFrame([
        (path.numpy().decode(), int(np.argmax(label.numpy())))
        for (image, label), path in ds
    ], columns=["path", "label"])

#znalezienie duplikatow:
def find_image_duplicates(**kwargs):
    datasets = list(kwargs.values())
    def image_hash(image):
      image_bytes = tf.io.serialize_tensor(image)
      return hashlib.md5(image_bytes.numpy()).hexdigest()
    if not datasets:
        return {
            "Liczba wszystkich obrazów:": 0,
            "Liczba duplikatów:": 0,
            "Unikalne obrazy:": 0,
            "Duplikaty szczegóły:": []
        }

    full_ds = datasets[0]
    for ds in datasets[1:]:
        full_ds = full_ds.concatenate(ds)

    hash_to_path = {}
    duplicates = []
    total_images = 0

    for (img, _), path in full_ds:
      total_images += 1
      h = image_hash(img)
      #print(h)
      path_str = path.numpy().decode()

      if h in hash_to_path:

          duplicates.append((path_str, hash_to_path[h]))
      else:

          hash_to_path[h] = path_str

    return {
        "Number of images": total_images,
        "Number of duplicates": len(duplicates),
        "Unique images": len(hash_to_path),
        "Duplicates details": duplicates
    }

#pokazanie rozdystrybuuowania klas w zbiorze:
def plot_class_distribution(df, column="label_name", title="Rozkład klas"):
    labels = df[column].value_counts().index.tolist()
    colors = sns.color_palette("Blues", n_colors=len(labels))

    df[column].value_counts(normalize=True).plot(
        kind="pie",
        autopct="%1.1f%%",
        labels=labels,
        colors=colors,
        title=title,
        ylabel=""
    )

    plt.show()

#usuniecie zduplikowanych obrazow:
def remove_paths_from_dataset(dataset, paths_to_remove):
    paths_to_remove = tf.constant(paths_to_remove)
    def keep_fn(data, path):
        is_equal = tf.reduce_any(tf.equal(paths_to_remove, path))
        return tf.logical_not(is_equal)

    return dataset.filter(keep_fn)


#pokazanie obrazow ze zbiorow:
def show_images(paths, rows=None, cols=None, figsize=(16, 8)):
    n = len(paths)
    if rows is None and cols is None:
        cols = min(5, n)
        rows = math.ceil(n / cols)
    elif rows is None:
        rows = math.ceil(n / cols)
    elif cols is None:
        cols = math.ceil(n / rows)

    plt.figure(figsize=figsize)

    for i, path in enumerate(paths, 1):
        img = Image.open(path)
        plt.subplot(rows, cols, i)
        plt.imshow(img)
        plt.axis("off")

    plt.tight_layout()
    plt.show()

#podzial datasetu na traningowy i testowy:
def split_train_test_ds(dataset, dataset_size, split_ratio=0.2, batch_size=32, limit_ds=None):
    dataset = dataset.shuffle(buffer_size=dataset_size)
    split_index = int(dataset_size * split_ratio)
    train_dataset = dataset.skip(split_index)
    #train_dataset = train_dataset.batch(batch_size)
    test_dataset = dataset.take(split_index)
    #test_dataset = test_dataset.batch(batch_size)
    if limit_ds is not None:
        train_ratio = sum(1 for _ in train_dataset)*limit_ds
        test_ratio = sum(1 for _ in test_dataset)*limit_ds
        train_dataset = train_dataset.take(train_ratio)
        test_dataset = test_dataset.take(test_ratio)
    return train_dataset, test_dataset


**Problem badawczy:** Zbudowanie i ocena modelu klasyfikacji obrazów MRI mózgu z wykorzystaniem sieci konwolucyjnej (CNN – Convolutional Neural Network). Model ma rozpoznawać rodzaj obrazu MRI spośród czterech kategorii:

*   Glioma tumor - Glejak
*   Meningioma tumor - Oponiak
*   Pituitary tumor - Guz przysadki mózgowej
*   No tumor - Brak guza (pacjent zdrowy)

Zadanie to jest przykładem klasyfikacji binominalnej obrazów medycznych.


Analiza obrazów MRI jest kluczowa w diagnostyce chorób mózgu, ale wymaga czasu i specjalistycznej wiedzy. Automatyczne modele oparte na uczeniu głębokim, takie jak CNN, mogą wspierać lekarzy w szybszym i bardziej obiektywnym wstępnym rozpoznaniu typu guza lub jego braku. Projekt ma na celu sprawdzenie, jak dobrze prosty model CNN radzi sobie z takim zadaniem.

Opis danych
W projekcie wykorzystano publiczny zbiór danych z platformy Kaggle: Brain Tumor MRI Dataset. Zbiór zawiera obrazy MRI mózgu podzielone na foldery odpowiadające czterem klasom. Dane są już rozdzielone na zbiór treningowy (Training) oraz testowy (Testing).

In [ ]:
#pobranie danych i wczytanie danych - dane laczymy by pozniej sprawdzic czy nie ma duplikatow
full_ds, categories, tr, ts = data_reader("masoudnickparvar/brain-tumor-mri-dataset")


In [ ]:
#sprawdzenie katergorii (etykiet)
categories

In [ ]:
#opis datasetu
full_ds

In [ ]:
#sprawdzenie reczne  etykiet w one hot encoding dla 5 pierwszych losowych etykiet
for (_, labels),_  in full_ds.shuffle(7200).take(5):
    print(labels)


In [ ]:
#sprawdzenie reczne tensorow obrazow dla 1 pierwszego obrazka
for (images, _),_  in full_ds.take(1):
    print(images)


<span style="color: lightblue">

### 2.  EDA - Explanatory data analysis

<span>

In [ ]:
#sprawdzenie czy mamy zduplikowane obrazy w datasecie - zbiory tetsowy i treningowy wczesniej zostaly polaczone w calosc by uniknac duplikatow
#pomiedzy zbiorem treningowym a testowym, ktore moglyby spowdowac przeuczenie sie

duplicates = find_image_duplicates(full_ds=full_ds)

In [ ]:
duplicates

In [ ]:
#wizualizacja przykladowych duplikatow:
show_images(['/kaggle/input/brain-tumor-mri-dataset/Training/meningioma/Tr-me_1242.jpg', '/kaggle/input/brain-tumor-mri-dataset/Training/meningioma/Tr-aug-me_90.jpg', '/kaggle/input/brain-tumor-mri-dataset/Training/meningioma/Tr-me_1274.jpg',
   '/kaggle/input/brain-tumor-mri-dataset/Training/meningioma/Tr-aug-me_98.jpg', '/kaggle/input/brain-tumor-mri-dataset/Testing/notumor/Te-no_92.jpg',
   '/kaggle/input/brain-tumor-mri-dataset/Training/notumor/Tr-no_1319.jpg', '/kaggle/input/brain-tumor-mri-dataset/Testing/notumor/Te-no_194.jpg',
   '/kaggle/input/brain-tumor-mri-dataset/Training/notumor/Tr-no_1004.jpg'], rows = 2, cols = 4)

In [ ]:
#Identyfikacja duplikatow do usuniecia
paths_to_remove = [e[0] for e in duplicates['Duplicates details']]

In [ ]:
#usuniecie duplikatow
deduplicated_ds = remove_paths_from_dataset(full_ds, paths_to_remove)

In [ ]:
#Ponowne sprawdzenie czy nie ma duplikatow w celu upewnienia sie, ze zostaly usuniete porpawnie:
final_stat = find_image_duplicates(deduplicated_ds=deduplicated_ds)
final_stat

In [ ]:
#podzial zbiory na zbior testowy i traningowy
train_ds, test_ds = split_train_test_ds(deduplicated_ds, final_stat['Number of images'], 0.3)

In [ ]:
#opis traningowego datasetu:
train_ds

In [ ]:
#opis testowego datasetu:
test_ds

In [ ]:
#zmiana datasetu treningowego na df:
train_df = dataset_to_df(train_ds)
train_df.head(100)

In [ ]:
#zmiana datasetu treningowego na df:
test_df = dataset_to_df(test_ds)
test_df.head(100)

In [ ]:
#wyciagniecie nazwy etykiety dla konkretnej labelki zbioru treningowego
train_df["label_name"] = train_df["path"].apply(lambda p: p.split("/")[-2])

In [ ]:
#pokazanie rozkladu klas w zbiorze treningowym
plot_class_distribution(train_df, "label_name", "Rozkład klas w zbiorze treningowym")

In [ ]:
#sprawdzenie czy balans rozkladu klas w zbiorze treningowym jest prawidlowo zrownowazony
check_balance(train_df, 'label_name')

In [ ]:
#wyciagniecie nazwy etykiety dla konkretnej labelki zbioru testowego
test_df["label_name"] = train_df["path"].apply(lambda p: p.split("/")[-2])

In [ ]:
#pokazanie rozkladu klas w zbiorze testowym
plot_class_distribution(test_df, "label_name", "Rozkład klas w zbiorze testowym")

In [ ]:
#sprawdzenie czy balans rozkladu klas w zbiorze testowym jest prawidlowo zrownowazony
check_balance(test_df, 'label_name')

**Wniosek ogolny:** Rozkład klas w zbiorze treningowym i testowym jest identyczny, co wskazuje na prawidłowy podział danych. Zbiory są  zbalansowane, co sprzyja stabilnemu trenowaniu modelu klasyfikacyjnego

In [ ]:
#Wizualizacja przykladowych obrazow:
sample = train_df.sample(n=15, random_state=42)
show_images(paths=sample["path"].tolist(), rows=3, cols=5)





**Wnioski:**


*   Kolorystyka obrazow rozni sie - czesc obrazow jest czarnobiala, a czesc w kolorach -> Potrzebna normalizacja
*   Jasnosc i kontrast sa zroznicowane nawet w ramach odcieni czerni - raz obraz jest jasniejszy, raz ciemniejszy -> Potrzebna normalizacja
*   Obrazy mają różne rozmiary -->  Potrzebny crop
*   Obrazy pochodzą z różnych płaszczyzn przekroju, rozne orientacje i skale --> Potrzebne filtry
*   Na obrazach widac napisy np pasek u gory zdjecia
*   Na obrazkach jest duzo tla







In [ ]:
train_ds2 = train_ds.map(
    lambda data, path: (tf.image.rgb_to_grayscale(data[0]), data[1])
)

test_ds2 = test_ds.map(
    lambda data, path: (tf.image.rgb_to_grayscale(data[0]), data[1])
)

In [ ]:
train_ds2 = train_ds2.map(
    lambda x, y: (tf.image.central_crop(x, 0.8), y)
)

test_ds2 = test_ds2.map(
    lambda x, y: (tf.image.central_crop(x, 0.8), y)
)

In [ ]:
train_ds2 = train_ds2.map(
    lambda x, y: (tf.image.resize(x, (150,150)), y)
)

test_ds2 = test_ds2.map(
    lambda x, y: (tf.image.resize(x, (150,150)), y)
)

In [ ]:
train_ds2 = train_ds2.map(
    lambda x, y: (tf.cast(x, tf.float32) / 255.0, y)
)

test_ds2 = test_ds2.map(
    lambda x, y: (tf.cast(x, tf.float32) / 255.0, y)
)

In [ ]:
train_ds2 = train_ds2.map(lambda x, y: (tf.image.grayscale_to_rgb(x), y))
test_ds2  = test_ds2.map(lambda x, y: (tf.image.grayscale_to_rgb(x), y))

In [ ]:
augment = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.05),
    tf.keras.layers.RandomZoom(0.1),
])

train_ds2 = train_ds2.map(lambda x,y: (augment(x), y))

In [ ]:
print(train_ds2.element_spec)

In [ ]:
#jakie inne operacje?

<span style="color: lightblue">

### 3. Model

<span>

In [ ]:
# unet i potem drugi model (czy mozna np xgboost na obrazki?)

In [ ]:
#tu zdecydowanie trzeba poprawic, wrzucilam tak o, nie myslalam o tym jeszcze

def unet_like_model():
    inputs = tf.keras.Input(shape=(150, 150, 3), dtype="float32")

    # Encoder
    x = tf.keras.layers.Conv2D(32, (3,3), activation="relu", padding="same")(inputs)
    x = tf.keras.layers.MaxPooling2D((2,2))(x)

    x = tf.keras.layers.Conv2D(64, (3,3), activation="relu", padding="same")(x)
    x = tf.keras.layers.MaxPooling2D((2,2))(x)

    # Bottleneck
    x = tf.keras.layers.Conv2D(128, (3,3), activation="relu", padding="same")(x)

    # Decoder (bardzo uproszczony)
    x = tf.keras.layers.UpSampling2D((2,2))(x)
    x = tf.keras.layers.Conv2D(64, (3,3), activation="relu", padding="same")(x)

      # Decoder (bardzo uproszczony)
    x = tf.keras.layers.UpSampling2D((2,2))(x)
    x = tf.keras.layers.Conv2D(32, (3,3), activation="relu", padding="same")(x)

    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    outputs = tf.keras.layers.Dense(4, activation="softmax")(x)

    return tf.keras.Model(inputs, outputs)

model = unet_like_model()
model.summary()


In [ ]:
model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"],
    #learning_rate=0.001
)



In [ ]:
train_ds2

In [ ]:
tr.batch(32)

In [ ]:
train_ds2

In [ ]:
batched_train_ds2 = train_ds2.batch(32)
batched_test_ds2 = test_ds2.batch(32)

In [ ]:
batched_train_ds2

In [ ]:
history = model.fit(
    batched_train_ds2,
    validation_data=batched_test_ds2,
    epochs=10
)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8,5))
plt.plot(history.history["accuracy"], label="train")
plt.plot(history.history["val_accuracy"], label="val")
plt.title("Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
loss, accuracy = model.evaluate(test_ds, verbose=0)

print(f"Dokładność na zbiorze testowym: {accuracy:.4f}")
print(f"Strata na zbiorze testowym: {loss:.4f}")

In [ ]:
#dodalabym confusion matrix (w postaci diagramu)

In [ ]:
#metryki: recall i precision,  accuracy, f1 score etc
#heatmapa do metryk
#moglibysmy tez zrobic wizualizacje - obrazek i pod spodem info czy trag=filismy czy nie, robilismy cos takiego z Kuba

<span style="color: lightblue">

### 4. Optymalizacja

<span>

In [ ]:
#tu optuna :) uzyjmy slajdow Natalii by zidentyfiukowac ktore parametry chcemy podrasowac (ostatnia przeka z architektury z zajec)

# Wypisać prametry do optymalizacji

<span style="color: lightblue">

### 5. Wnioski

<span>

In [ ]:
#wylistowac wnioski
#sprawdzic w chat i w internecie czy jest do obrazkow cos na podobienstwo shapa i jesli tak to uzyc